### FM算法简介

FM算法全称Factorization Machines，是一种针对预测问题的通用范式，在CTR、CVR预估中有非常广泛的应用。FM算法有以下两点优势：

 - 适合高维稀疏特征，针对高维稀疏特征组合问题效果较好
 - 推理上消耗线性时间复杂度，不需要额外的数据存储（如SVM的支持向量点需要额外存储）

在模型层面进行特征组合主要有两大类方法：FM算法和基于树的模型。FM算法使用分解的方法，对特征交叉进行建模，因此可以估计经过one-hot编码后高维稀疏特征之间的相互影响。

### FM模型形式

令$x$和$y$为模型的输入输出，$m(x)$为$x$的$l_0$范数，$\overline{m}(x)$为$m(x)$在数据集$D$上的平均值

首先给出二阶多项式模型，这种模型利用参数$w_{ij}$建模特征之间的两两交叉：

$y(x)=w_0+\sum_{i=1}^{n}w_ix_i+\sum_{i=1}^{n}\sum_{j=i+1}^{n}w_{ij}x_ix_j$

多项式模型的前半部分和LR是一致的，后半部分给出了二阶特征交叉的建模方法，其中任意两个不同的$w_{ij}$都是独立的，学习参数$w_{ij}$要求$x_i$和$x_j$同时不为0，而这样的样本非常少，对于高维稀疏特征来说参数的学习将会变得非常困难。

FM算法给出了改进，将$w_{ij}$以矩阵乘法的形式进行分解：

$y(x)=w_0+\sum_{i=1}^{n}w_ix_i+\sum_{i=1}^{n}\sum_{j=i+1}^{n}<v_i,v_j>x_ix_j$

其中需要估计的参数有下面这几个：$w_0\in{\mathbb{R}}$、$w\in{\mathbb{R^n}}$、$V\in{\mathbb{R}}^{n\times{k}}$，$k$是定义分解维度的超参数，其中$v_i$为$V$中的一行，也就是第$i$个特征的隐向量，$v_i$用$k$个因子描述了第$i$个特征

$\hat{w}_{ij}=<v_i,v_j>$对第$i$个特征和第$j$个特征的交叉进行建模，$V$矩阵仅仅包含$O(kn)$的存储空间占用，而并不是采用一个单独的变量$\hat{w}_{ij}$（这样会占用$n\times{n}$的存储空间）

$k$一般取4，FM算法的隐向量$v_i$可以看做是特征的一种embedding表示，把高维稀疏离散特征转化为稠密特征，这种dense feature可作为DNN的输入。

### 表达能力

这里有一个引理，对于任意的正定矩阵$W$，存在矩阵$V$使得$W=VV^\top$（$k$足够大），这意味着当$k$足够大的时候，FM算法可以表达任何特征交叉的矩阵$W$。对于高维稀疏特征，$k$最好取小一点，因为没有足够的数据对复杂的$W$进行估计，此时限制$k$可以使得FM获得更好的泛化能力，从而更好的估计$W$

### 对于稀疏数据的参数估计

稀疏情况下，通常没有足够的数据对特征之间的交互进行建模，但FM算法通过分解交互参数$w_{ij}$，对交互的独立性问题进行了解决，也就是说$w_{ij}=<v_i,v_j>$，把$w_{ij}$分解成了两个向量的点乘，只要存在$x_i$不为0，就可以更新$v_i$，而并不需要强制要求$x_i$和$x_j$均不为0，也就是说对于数据中没见到过的特征交互，FM算法依然可以完成估计；并且$V$这个隐向量可以表达没出现过的交叉特征，而多项式模型假设$w_{ij}$是相互独立的

### 模型复杂度

咋看起来，FM算法的时间复杂度是$O(kn^2)$，实际上的时间复杂度只需要$O(kn)$，这里对特征交互项进行了一个化简：

$\sum_{i=1}^{n}\sum_{j=i+1}^{n}<v_i,v_j>x_ix_j=\dfrac{1}{2}\sum_{f=1}^k((\sum_{i=1}^nv_{i,f}x_i)^2-\sum_{i=1}^nv_{i,f}^2x_i^2)$

从上面这个化简来看，特征交互这一项占用的时间复杂度节省到了$O(kn)$；对于稀疏特征来说，FM算法的时间复杂度是$O(k\overline{m}(x))$

### 学习算法

可以用基于梯度的方法对FM算法进行求解：

$\dfrac{\partial}{\partial{w_0}}y(x)=1$

$\dfrac{\partial}{\partial{w_i}}y(x)=x_i$

$\dfrac{\partial}{\partial{v_{i,f}}}y(x)=x_i\sum_{j=1}^{n}v_{j,f}x_j-v_{i,f}x_i^2$

由上面的公式可以知道，$v_{i,f}$的训练只需要样本的$x_i$特征非0即可，适合稀疏数据；

在每次SGD迭代中，对于所有的$v_{i,f}$，$\sum_{j=1}^{n}v_{j,f}x_j$独立于$i$，因此可以被预先计算，只需要算一次。因此$v_{i,f}$梯度计算的时间复杂度为$O(1)$，$v_{i,f}$一共有$kn$个，因此对于$(x,y)$，所有参数更新可以在$O(kn)$（或者$O(km(x))$）时间内完成

### 参考文献

- https://www.csie.ntu.edu.tw/~b97053/paper/Rendle2010FM.pdf
- https://zhuanlan.zhihu.com/p/37963267
- https://blog.csdn.net/asd136912/article/details/78318563